In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
from scipy.signal import savgol_filter, medfilt, find_peaks
from scipy.ndimage import binary_closing, binary_dilation
from scipy.signal.windows import triang
from scipy.ndimage import convolve1d
from tqdm.notebook import tqdm
from typing import List

### Preprocessing functions

In [2]:
def preprocess_timeseries(ts_df: pd.DataFrame) -> pd.DataFrame:
    """Preprocess timeseries data to correct timestamp format and more memory-efficient
    data types"""
    ts_df["timestamp"] = ts_df["timestamp"].str[:-5]  # remove tzinfo
    ts_df["timestamp"] = pd.to_datetime(ts_df["timestamp"], format="ISO8601")
    return ts_df


def preprocess_events(events_df: pd.DataFrame) -> pd.DataFrame:
    """Preprocess events data to correct timestamp format and more memory-efficient
    data types"""
    events_df = events_df.dropna().reset_index(drop=True)
    events_df["timestamp"] = events_df["timestamp"].str[:-5]  # remove tzinfo
    events_df["timestamp"] = pd.to_datetime(events_df["timestamp"], format="ISO8601")
    events_df["step"] = events_df["step"].astype(int)
    return events_df


def single_series_1min_resampling(ts_df: pd.DataFrame) -> pd.DataFrame:
    """Resample a single raw time series (5sec) data to 1min intervals"""
    sid = ts_df.series_id.iloc[0]
    ts_5sec_df = ts_df.copy()
    ts_5sec_df = ts_5sec_df.set_index("timestamp")
    ts_1min_df = ts_5sec_df.resample("1min").agg(
        step=("step", "min"),
        anglez_1min_mean=("anglez", "mean"),
        enmo_1min_mean=("enmo", "mean"),
    )
    contains_nan = ts_1min_df.isnull().values.any()
    if contains_nan:
        prev_len = len(ts_1min_df)
        ts_1min_df = ts_1min_df.dropna()
        new_len = len(ts_1min_df)
        print(f"Dropped {prev_len - new_len} entries in {sid}")
    ts_1min_df = ts_1min_df.reset_index()
    ts_1min_df["series_id"] = sid
    ts_1min_df["series_id"] = ts_1min_df["series_id"].astype("large_string[pyarrow]")
    ts_1min_df["hour"] = ts_1min_df["timestamp"].dt.hour.astype("uint8[pyarrow]")
    return ts_1min_df


def resample_timeseries_data_to_1min(series_df: pd.DataFrame) -> pd.DataFrame:
    """Iterate over each series in the data and resample to 1min interval"""
    series_grps = series_df.groupby(by="series_id")
    one_min_dfs = []
    for series_id, ts_df in tqdm(series_grps):
        ts_5sec_df = ts_df.copy()
        ts_5sec_df = preprocess_timeseries(ts_5sec_df)
        ts_1min_df = single_series_1min_resampling(ts_5sec_df)
        one_min_dfs.append(ts_1min_df)

    full_1min_df = pd.concat(one_min_dfs).reset_index(drop=True)

    return full_1min_df


def create_asleep_column(ts_df: pd.DataFrame, events_df: pd.DataFrame) -> pd.DataFrame:
    """Create a new binary column in the timeseries data indicating whether the person
    is asleep at the specified timestamp or not. This column can be derived from the
    onset and wakeup events in the events dataframe"""
    ts_df = ts_df.copy()
    ts_df["asleep"] = 0
    ts_df = ts_df.reset_index(drop=True)
    for night_id, night_df in events_df.groupby(by="night"):
        if len(night_df) != 2:
            continue
        onset_step = night_df[night_df["event"] == "onset"].step.item()
        wakeup_step = night_df[night_df["event"] == "wakeup"].step.item()
        step_list = ts_df["step"].tolist()
        onset_1min_idx = step_list.index(onset_step)
        wake_1min_idx = step_list.index(wakeup_step)
        ts_df.loc[onset_1min_idx:wake_1min_idx, "asleep"] = 1
    ts_df["asleep"] = ts_df["asleep"].astype("uint8[pyarrow]")
    return ts_df


def event_minmax_prune_timeseries(
    ts_df: pd.DataFrame, events_df: pd.DataFrame, hour_buffer: int
) -> pd.DataFrame:
    """Prune timeseries dataframe to only include timestamps in the min and max
    intervals of the event dataframe (some timeseries have long stretches of unannotated
    data)"""
    time_buffer = dt.timedelta(hours=hour_buffer)
    min_event_dt = events_df["timestamp"].min() - time_buffer
    max_event_dt = events_df["timestamp"].max() + time_buffer
    prune_idx = (ts_df["timestamp"] >= min_event_dt) & (
        ts_df["timestamp"] <= max_event_dt
    )
    ts_df = ts_df[prune_idx].reset_index(drop=True)
    return ts_df


def remove_no_annotation_periods(
    ts_df: pd.DataFrame, min_hours: int = 24
) -> pd.DataFrame:
    """Remove any sequences in the time series where no sleep has been detected for
    'min_hours'. This most likely indicates that data was not annotated in this
     period."""
    no_sleep_24h = (1 - maximum_filter1d(ts_df["asleep"], size=60 * min_hours)).astype(
        bool
    )
    ts_df = ts_df[~no_sleep_24h]
    return ts_df


def add_asleep_target_to_timeseries_data(
    series_df: pd.DataFrame, events_df: pd.DataFrame, remove_unannotated: bool = False
):
    """For each series in the data add a column which indicates whether the person is
    asleep or not. This is based on the onset and wakeup times in the events data."""
    series_grps = series_df.groupby(by="series_id")
    event_grps = events_df.groupby(by="series_id")
    series_grp_keys = set(series_grps.groups.keys())
    event_grp_keys = set(event_grps.groups.keys())
    # remove keys that are not present in both events and series data
    grp_keys = series_grp_keys.intersection(event_grp_keys)

    ts_asleep_dataframes = []
    for series_id in tqdm(grp_keys):
        event_df = event_grps.get_group(series_id)
        ts_df = series_grps.get_group(series_id)
        ts_df = create_asleep_column(ts_df, event_df)
        if remove_unannotated:
            ts_df = remove_no_annotation_periods(ts_df)
        ts_asleep_dataframes.append(ts_df)
    ts_asleep_df = pd.concat(ts_asleep_dataframes).reset_index(drop=True)
    return ts_asleep_df

### Feature_extraction functions

In [3]:
def iqr90(x):
    return np.quantile(x, q=0.95) - np.quantile(x, q=0.05)


def rolled_timeseries_features(
    ts_df: pd.DataFrame,
    agg_dict: dict,
    roll_freqs: List[str],
) -> pd.DataFrame:
    """Generate time series features based on rolling windows. In this case the
    mean and standard deviation of each 'roll_freqs' frequency will be calculated"""
    rolled_dfs = []
    for freq in roll_freqs:
        ts_roll_df = ts_df.rolling(freq).agg(agg_dict)
        feat_cols = [
            "_".join(name_parts) + f"_{freq}"
            for name_parts in ts_roll_df.columns.to_flat_index()
        ]
        ts_roll_df.columns = feat_cols
        ts_roll_df[feat_cols] = ts_roll_df[feat_cols].astype(np.float32)
        rolled_dfs.append(ts_roll_df)
    roll_df = pd.concat(rolled_dfs, axis=1)
    return roll_df


def extract_rolling_features(
    ts_1min_df: pd.DataFrame, agg_dict, roll_freqs
) -> pd.DataFrame:
    series_grps = ts_1min_df.groupby(by="series_id")
    feature_dataframes = []
    for series_id, ts_df in tqdm(series_grps):
        ts_df = ts_df.set_index("timestamp")
        roll_feat_df = rolled_timeseries_features(ts_df, agg_dict, roll_freqs)
        feat_df = pd.concat([ts_df, roll_feat_df], axis=1)
        feat_df = feat_df.dropna()
        feature_dataframes.append(feat_df)

    full_feat_df = pd.concat(feature_dataframes).reset_index(names="timestamp")
    full_feat_df["step"] = full_feat_df["step"].astype(np.uint32)

    return full_feat_df

### Load train and test data

In [4]:
train_datapath = "/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet"
raw_train_df = pd.read_parquet(train_datapath)

In [5]:
test_datapath = "/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet"
raw_test_df = pd.read_parquet(test_datapath)

### Resample data to 1min instead of 5sec frequency

In [6]:
raw_train_df["enmo"] = raw_train_df["enmo"].clip( 0.0, 3.0)
ts_train_1min_df = resample_timeseries_data_to_1min(raw_train_df)

  0%|          | 0/277 [00:00<?, ?it/s]

Dropped 60 entries in 1087d7b0ff2e
Dropped 60 entries in 18a0ca03431d
Dropped 60 entries in 292a75c0b94e
Dropped 60 entries in 2f7504d0f426
Dropped 60 entries in 7476c0bd18d2
Dropped 60 entries in 808652a666c6
Dropped 60 entries in 844f54dcab89
Dropped 60 entries in aa81faa78747
Dropped 60 entries in b4b75225b224
Dropped 60 entries in b737f8c78ec5
Dropped 60 entries in b84960841a75
Dropped 60 entries in bccf2f2819f8
Dropped 60 entries in bfe41e96d12f
Dropped 60 entries in c3072a759efb
Dropped 60 entries in c5d08fc3e040
Dropped 60 entries in c75b4b207bea
Dropped 60 entries in dfc3ccebfdc9


In [7]:
raw_test_df["enmo"] = raw_test_df["enmo"].clip( 0.0, 3.0)
ts_test_1min_df = resample_timeseries_data_to_1min(raw_test_df)

  0%|          | 0/3 [00:00<?, ?it/s]

### Add binary target column that indicates whether people are asleep

In [8]:
events_path = "/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv"
raw_events_df = pd.read_csv(events_path)

In [9]:
events_df = raw_events_df.copy()
events_df = preprocess_events(events_df)

In [10]:
ts_train_1min_df = add_asleep_target_to_timeseries_data(ts_train_1min_df, events_df)

  0%|          | 0/269 [00:00<?, ?it/s]

### Extract rolling features

In [11]:
def calculate_first_order_variation(ts_1min_df, var_col):
    first_order_var = ts_1min_df.groupby(by="series_id")[var_col].diff().abs()
    first_order_var[first_order_var.isna()] = 0
    return first_order_var

In [12]:
ts_train_1min_df["anglez_1min_1v"] = calculate_first_order_variation(ts_train_1min_df, "anglez_1min_mean")
ts_train_1min_df["enmo_1min_1v"] = calculate_first_order_variation(ts_train_1min_df, "enmo_1min_mean")
ts_test_1min_df["anglez_1min_1v"] = calculate_first_order_variation(ts_test_1min_df, "anglez_1min_mean")
ts_test_1min_df["enmo_1min_1v"] = calculate_first_order_variation(ts_test_1min_df, "enmo_1min_mean")

In [13]:
awake_df = ts_train_1min_df[ts_train_1min_df["asleep"]==0]
awake_enmo_median = awake_df["enmo_1min_mean"].median()
ts_train_1min_df["awake_count"] = ts_train_1min_df["enmo_1min_mean"]>awake_enmo_median
ts_test_1min_df["awake_count"] = ts_test_1min_df["enmo_1min_mean"]>awake_enmo_median

In [14]:
roll_freqs = ["15min", "30min", "1H"]
agg_funcs = ["mean", "std", "max"]
agg_dict = {
    "enmo_1min_mean": agg_funcs,
    "enmo_1min_1v": agg_funcs,
    "anglez_1min_mean": agg_funcs,
    "anglez_1min_1v": agg_funcs,
    "awake_count": "mean"
}

In [15]:
ts_train_feat_df = extract_rolling_features(ts_train_1min_df, agg_dict, roll_freqs)

  0%|          | 0/269 [00:00<?, ?it/s]

In [16]:
ts_test_feat_df = extract_rolling_features(ts_test_1min_df, agg_dict, roll_freqs)

  0%|          | 0/3 [00:00<?, ?it/s]

### Put extra emphasis on transition periods

In [17]:
def rectangular_transition_sample_weights(sample_weights, transition_idxs, transition_weight, window_size=31):
    transition_filter = np.zeros_like(sample_weights)
    transition_filter[transition_idxs] = 1
    transition_filter = binary_dilation(transition_filter, structure=np.ones(window_size))
    transition_filter = transition_filter*transition_weight
    sample_weights += transition_filter
    return sample_weights

In [18]:
def triangular_transition_sample_weights(sample_weights, transition_idxs, transition_weight, window_size=31):
    transition_filter = np.zeros_like(sample_weights)
    transition_filter[transition_idxs] = 1
    tri_window = triang(window_size)*transition_weight
    transition_filter = convolve1d(transition_filter, tri_window)
    sample_weights += transition_filter
    return sample_weights

In [19]:
transition_idxs = ts_train_feat_df.reset_index().merge(events_df, on=["step","series_id"])["index"].values
sample_weights = np.ones(len(ts_train_feat_df))
sample_weights = triangular_transition_sample_weights(sample_weights, transition_idxs, transition_weight=10, window_size=31)

### Train model

In [20]:
feat_cols = ts_train_feat_df.columns.drop(["timestamp", "series_id", "step", "asleep", "awake_count"]).tolist()
target_col = "asleep"
X_train = ts_train_feat_df[feat_cols].copy()
y_train = ts_train_feat_df[target_col].copy()

In [21]:
clf = CatBoostClassifier(n_estimators=500, cat_features=["hour"], verbose=10)
clf.fit(X_train, y_train, sample_weight=sample_weights)
clf.score(X_train, y_train)

Learning rate set to 0.5
0:	learn: 0.3048957	total: 7.37s	remaining: 1h 1m 16s
10:	learn: 0.2065311	total: 1m 9s	remaining: 51m 44s
20:	learn: 0.1972606	total: 2m 11s	remaining: 49m 52s
30:	learn: 0.1915895	total: 3m 9s	remaining: 47m 42s
40:	learn: 0.1872777	total: 4m 12s	remaining: 47m 5s
50:	learn: 0.1839539	total: 5m 10s	remaining: 45m 30s
60:	learn: 0.1814005	total: 6m 10s	remaining: 44m 26s
70:	learn: 0.1790665	total: 7m 12s	remaining: 43m 32s
80:	learn: 0.1770852	total: 8m 14s	remaining: 42m 39s
90:	learn: 0.1749012	total: 9m 15s	remaining: 41m 36s
100:	learn: 0.1734194	total: 10m 17s	remaining: 40m 40s
110:	learn: 0.1720039	total: 11m 17s	remaining: 39m 34s
120:	learn: 0.1706553	total: 12m 17s	remaining: 38m 31s
130:	learn: 0.1692163	total: 13m 19s	remaining: 37m 33s
140:	learn: 0.1676340	total: 14m 22s	remaining: 36m 37s
150:	learn: 0.1667162	total: 15m 22s	remaining: 35m 32s
160:	learn: 0.1658010	total: 16m 22s	remaining: 34m 28s
170:	learn: 0.1648549	total: 17m 23s	remaining

0.9661553653941106

### Make predictions

In [22]:
def filter_predictions(y_proba, window_size):
    y_proba_smooth = savgol_filter(y_proba, window_length=window_size, polyorder=2)
    y_pred = (y_proba_smooth > 0.5)*1
    if (window_size%2)==0: window_size+=1 
    y_pred = binary_closing(y_pred, structure=np.ones(window_size))
    return y_pred

In [23]:
def score_sleep_periods(y_proba, onset_idxs, wakeup_idxs):
    onset_scores = []
    wakeup_scores = []
    for start_idx, end_idx in zip(onset_idxs, wakeup_idxs):
        yp = y_proba[start_idx:end_idx]
        lin_weight = np.linspace(0, 1, num=len(yp))
        onset_score_ = np.sum(yp*lin_weight[::-1])/np.sum(lin_weight)
        wakeup_score_ = np.sum(yp*lin_weight)/np.sum(lin_weight)
        onset_scores.append(onset_score_)
        wakeup_scores.append(wakeup_score_)
    return np.array(onset_scores), np.array(wakeup_scores)

In [24]:
def create_event_dataframe(feat_df, event_idxs, event_scores, event_type):
    event_df = feat_df.loc[event_idxs, ["step", "series_id"]]
    event_df["event"] = event_type
    event_df["score"] = event_scores
    return event_df

In [25]:
window_size = 12 # mins
plateau_size = 120 # mins
test_event_dfs = []
series_grps = ts_test_feat_df.groupby(by="series_id")
for series_id, test_feat_df in tqdm(series_grps):
    test_feat_df = test_feat_df.reset_index(drop=True)
    X_test = test_feat_df[feat_cols].copy()
    y_proba = clf.predict_proba(X_test)
    y_proba = y_proba[:, 1]
    y_pred = filter_predictions(y_proba, window_size)
    peaks, plateaus = find_peaks(y_pred, plateau_size=plateau_size)

    # get indices of onset and wake ups
    onset_idxs = plateaus["left_edges"]
    wakeup_idxs = plateaus["right_edges"]

    # score each sleeping event and create event dataframe
    onset_scores, wakeup_scores = score_sleep_periods(y_proba, onset_idxs, wakeup_idxs)
    onset_df = create_event_dataframe(
        test_feat_df, onset_idxs, onset_scores, event_type="onset"
    )
    wakeup_df = create_event_dataframe(
        test_feat_df, wakeup_idxs, wakeup_scores, event_type="wakeup"
    )
    event_df = pd.concat([onset_df, wakeup_df])
    event_df = event_df.sort_values(by="step")
    test_event_dfs.append(event_df)

submission_df = (
    pd.concat(test_event_dfs).reset_index(drop=True).reset_index(names="row_id")
)

  0%|          | 0/3 [00:00<?, ?it/s]

In [26]:
submission_df = pd.concat(test_event_dfs).reset_index(drop=True).reset_index(names="row_id")
submission_df.to_csv("submission.csv", index=False)

In [27]:
submission_df

,row_id,step,series_id,event,score
